In [1]:
# one_d_maze_abstract_time_to_criterion_control_final
import keras
import hrr as hrr
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
bias = 1.0
reward = 1.0
gamma = 0.9
alpha = 0.01
lambd = 0.0
states = 20
non_reward = 0
hrr_size = 2048
non_goal_states = 0
gates = ['open', 'close']
actions = ['left', 'right']
colors = ['red', 'green', 'purple', 'blue']

runs = 10
ep_down_1 = 3.0
ep_down_2 = 2.0
ep_down_3 = 1.0
fold_counter_ar = []
len_memory_store = []
sub_optimal_goal = 0.5
last_100_sub_optimal_runs = []

In [3]:
def problem(prob):
    rand = np.random.random()
    if sum(prob) > 1:
        print('ERROR CHECK PROBLEM FUNCTION')
    mem = prob[0]
    color = ''
    for i in range(len(prob)):
    #     print(i, "mem =", mem)
        if rand < mem:
            color = colors[i]
            break
        else:
            mem += prob[i+1]
    return color

In [4]:
# working memory
# this will choose the open/close action

def encode_gate(state, signal, w_m, gate_action):
    string = 'state_'  + str(state) + "*" + \
             'signal_' + str(signal) + "*" + \
             'w_m_' + str(w_m) + "*" + \
             'gate_action' + str(gate_action)
    encoded = np.array([memory.encode(string)])
    return(encoded)

def q_gate(state, signal, w_m):
    max_value = 0
    
    if signal != '':
        signal = 'on'        
    if w_m != '':
        w_m = 'on'
        
    for gate_action in gates:
        hrr = encode_gate(state, signal, w_m, gate_action)
        value = model_gate.predict(hrr)[0,0] + bias
        if value > max_value:
            max_value = value
            max_hrr = hrr
            max_gate = gate_action
    return  max_value, max_hrr, max_gate

In [5]:
# working memory
# move l/r

def encode_move(state, signal, w_m, phyical_action):
    string = 'state_'  + str(state) + "*" + \
             'signal_' + str(signal) + "*" + \
             'w_m_' + str(w_m) + "*" + \
             'phyical_action' + str(phyical_action)
    encoded = np.array([memory.encode(string)])
    return(encoded)

def q_move(state, signal, w_m):
    max_value = 0
    for phyical_action in actions:
        hrr = encode_move(state, signal, w_m, phyical_action)
        value = model_move.predict(hrr)[0,0] + bias
        if value > max_value:
            max_value = value
            max_hrr = hrr
            max_action = phyical_action
    return  max_value, max_hrr, max_action

In [6]:
# random mover

def random_q_gate(state, signal, w_m):
    if signal != '':
        signal = 'on'        
    if w_m != '':
        w_m = 'on'

    gate = np.random.choice(gates)
    hrr = encode_gate(state, signal, w_m, gate)
    value = model_gate.predict(hrr)[0,0] + bias
    return value, hrr, gate

def random_q_move(state, signal, w_m):
    phyical_move = np.random.choice(actions)
    hrr = encode_move(state, signal, w_m, phyical_move)
    value = model_move.predict(hrr)[0,0] + bias
    return value, hrr, phyical_move

In [7]:
for run in range(runs):

    episode = 0
    epsilon = 0.05
    fold_counter = 0
    # if time_to_cry = True   code will skip pre-training
    # if time_to_cry = False  code will      pre-train
    time_to_cry = True
#     time_to_cry = False
    
    ep_one = True
    ep_two = True
    ep_tre = True
    ep_back_on = False
    run_episode = True    
    sub_optimal_steps = []
    sub_optimal_average = []
    fold_sub_optimal_steps = []
    fold_sub_optimal_average = []
    
    memory = hrr.LTM(hrr_size, True)

    model_gate = keras.models.Sequential()
    model_gate.add(keras.layers.Dense(1,input_shape=[hrr_size],use_bias=False))
    model_gate.compile(loss=keras.losses.mse,
                       optimizer=keras.optimizers.SGD(learning_rate=alpha),
                       metrics=['accuracy'])
    model_gate.summary()

    model_move = keras.models.Sequential()
    model_move.add(keras.layers.Dense(1,input_shape=[hrr_size],use_bias=False))
    model_move.compile(loss=keras.losses.mse,
                       optimizer=keras.optimizers.SGD(learning_rate=alpha),
                       metrics=['accuracy'])
    model_move.summary()

    colorar = []

    while run_episode == True:
        
        fold = False

        if (episode+1) % 1000 == 0:
            print("on ", episode+1)
            print('reached ', sub_optimal_average[-1])

        if len(sub_optimal_average) > 0:
            if ep_tre == True:
                if ep_one == True:
                    if sub_optimal_average[-1] < ep_down_1: 
                        epsilon = epsilon/2
                        ep_one = False
                        print('one epsilon changed to:', epsilon)
                        print('reached ', sub_optimal_average[-1])

                elif ep_two == True:
                    if sub_optimal_average[-1] < ep_down_2: 
                        epsilon = epsilon/2
                        ep_two = False
                        print('two epsilon changed to:', epsilon)
                        print('reached ', sub_optimal_average[-1])
                else:
                    if sub_optimal_average[-1] < ep_down_3:
                        epsilon = epsilon/2
                        ep_tre = False
                        print('tre epsilon changed to:', epsilon)
                        print('reached ', sub_optimal_average[-1])

        current_state = np.random.choice(states)
        
        if time_to_cry == False:
            if len(sub_optimal_average) > 0:
                if sub_optimal_average[-1] < sub_optimal_goal:
                    time_to_cry = True
                    print('reached ', sub_optimal_average[-1], 'switch to crying')
                    print('location', len(sub_optimal_average))
                    ep_back_on = True
                    
        if ep_back_on == True:
            if sub_optimal_average[-1] > 4.0:
                epsilon = 0.05
                print('epsilon back on to ', epsilon)
                ep_one = True
                ep_two = True
                ep_tre = True
                ep_back_on = False
            
        if time_to_cry == False:
            prob = [1/2, 1/2, 0, 0]
        else:
            prob = [0, 0, 1/2, 1/2]
            
        current_signal = problem(prob)
        colorar.append(current_signal)
        
        if current_signal == 'purple':
            fold = True
        if current_signal == 'blue':
            fold = True

        if current_signal == 'red':
            goal_state = 2
        elif current_signal == 'green':
            goal_state = 7
        elif current_signal == 'purple':
            goal_state = 12
        else:
            goal_state = 17

        if goal_state < current_state:
            optimal_steps_1 = abs(goal_state - current_state)
            optimal_steps_2 = states - current_state + goal_state
        else:
            optimal_steps_1 = abs(goal_state - current_state)
            optimal_steps_2 = states - goal_state + current_state

        if optimal_steps_1 < optimal_steps_2:
            optimal_steps = optimal_steps_1
        else:
            optimal_steps = optimal_steps_2

        current_w_m = ''

        for step in range(states):

            if current_state == goal_state:
                break

            if np.random.random() < epsilon:
                gate_value, \
                gate_hrr, \
                gate_action = random_q_gate(current_state, current_signal, current_w_m)
                current_value, \
                current_hrr, \
                current_action = random_q_move(current_state, current_signal, current_w_m)         
            else:
                gate_value, \
                gate_hrr, \
                gate_action = q_gate(current_state, current_signal, current_w_m)
                current_value, \
                current_hrr, \
                current_action = q_move(current_state, current_signal, current_w_m)

            previous_state = current_state

            previous_gate_value = gate_value
            previous_gate_hrr = gate_hrr
            previous_gate_action = gate_action

            previous_value = current_value
            previous_hrr = current_hrr
            previous_action = current_action

            if gate_action == 'open':
                current_w_m = current_signal

            current_signal = ''

            if (current_action == "left"):
                current_state = (current_state-1)%states
            else:
                current_state = (current_state+1)%states

            if current_state == goal_state:
                target = reward
            else:
                current_value, _, _ = q_move(current_state, current_signal, current_w_m)
                target = non_goal_states + gamma * current_value

            model_gate.fit(previous_gate_hrr, [target-bias], verbose=0)
            model_move.fit(previous_hrr, [target-bias], verbose=0)

        sub_optimal_steps.append(abs(step-optimal_steps))
        
        if fold == True:
            if (episode+1) % 1000 == 0:
                print('fold reached ', fold_sub_optimal_average[-1])
            fold_counter += 1
            fold_sub_optimal_steps.append(abs(step-optimal_steps))
            
#             if len(fold_sub_optimal_steps) < 100:
#                 fold_sub_optimal_average.append(sum(fold_sub_optimal_steps)/len(fold_sub_optimal_steps))
#             else:
#                 fold_sub_optimal_average.append(sum(fold_sub_optimal_steps[-100:])/100)

            if len(fold_sub_optimal_steps) > 99:
                fold_sub_optimal_average.append(sum(fold_sub_optimal_steps[-100:])/100)
                print('number of purple and blue episodes: ', fold_counter)
                break
#                 if fold_sub_optimal_average[-1] < sub_optimal_goal:
#                     fold_counter_ar.append(fold_counter)
#                     print('number of purple and blue episodes: ', fold_counter)
#                     break
                
#         if len(sub_optimal_steps) < 100:
#             sub_optimal_average.append(sum(sub_optimal_steps)/len(sub_optimal_steps))
#         else:
#             sub_optimal_average.append(sum(sub_optimal_steps[-100:])/100)

        if len(sub_optimal_steps) > 99:
            sub_optimal_average.append(sum(sub_optimal_steps[-100:])/100)

        episode += 1

    list_freq = (Counter(colorar))
    
    for key, value in list_freq.items():
        print(key, " has count ", value)
    
    len_memory_store.append(len(memory.store))
    print('size of dictionary ', len(memory.store))
    
    last_100_sub_optimal_runs.append(fold_sub_optimal_average[0])
    print('100 runs average: ', fold_sub_optimal_average)
    
#     plt.plot(sub_optimal_average)
#     plt.grid(True)
#     plt.show()
    
#     plt.plot(fold_sub_optimal_average)
#     plt.grid(True)
#     plt.show()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 2048      
Total params: 2,048
Trainable params: 2,048
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 2048      
Total params: 2,048
Trainable params: 2,048
Non-trainable params: 0
_________________________________________________________________
number of purple and blue episodes:  100
purple  has count  57
blue  has count  43
size of dictionary  1002
100 runs average:  [9.71]
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

number of purple and blue episodes:  100
purple  has count  42
blue  has count  58
size of dictionary  1118
100 runs average:  [9.35]
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 1)                 2048      
Total params: 2,048
Trainable params: 2,048
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 1)                 2048      
Total params: 2,048
Trainable params: 2,048
Non-trainable params: 0
_________________________________________________________________
number of purple and blue episodes:  100
blue  has count  49
purple  has count  51
size of dictionary  948
100 runs average:  [9.47]
Model: "sequential_19"


In [8]:
len_memory_store

[1002, 1044, 1062, 1044, 1076, 1030, 1066, 1118, 948, 1002]

In [9]:
last_100_sub_optimal_runs

[9.71, 9.3, 10.27, 9.02, 10.82, 8.54, 9.58, 9.35, 9.47, 9.96]

In [10]:
stan_error = np.std(last_100_sub_optimal_runs)/np.sqrt(len(last_100_sub_optimal_runs))

In [11]:
mean = np.mean(last_100_sub_optimal_runs)
mean

9.602

In [12]:
upper_limit = mean + (1.96 * stan_error)
upper_limit

9.979459566258427

In [13]:
lower_limit = mean - (1.96 * stan_error)
lower_limit

9.224540433741574

In [15]:
(1.96 * stan_error)

0.37745956625842736